# Automated ML

In this notebook, I will perform the following steps:

Import Dependencies
Create an Experiment in an existing Workspace.
Create or Attach existing AmlCompute to a workspace.
Define data loading in a TabularDataset.
Configure AutoML using AutoMLConfig.
Use AutoMLStep
Train the model using AmlCompute
Explore the results.
Test the best fitted model.

## Import Dependcies

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [41]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources
import json
import pickle

import azureml.core
from azureml.core import Model
from azureml.core.environment import Environment
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep
from azureml.widgets import RunDetails
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.model import Model
from azureml.core.webservice import LocalWebservice
from azureml.pipeline.core import PipelineData, TrainingOutput
from azureml.core.model import Model

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

import urllib.request


import joblib



SDK version: 1.39.0


## Create an Azure ML experiment

In [5]:
ws = Workspace.from_config()

# Choose a name for the run history container in the workspace.
experiment_name = 'ml-experiment-1'
project_folder = './automl-project'

experiment = Experiment(ws, experiment_name)
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-191197
aml-quickstarts-191197
southcentralus
6971f5ac-8af1-446e-8034-05acea24681f


## Create or Attach an AmlCompute cluster

In [6]:
# Choose a name for your CPU cluster
amlcompute_cluster_name = "automl-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Found existing cluster, use it.
Succeeded.....................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [7]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "Heart_Prediction_Dataset"
description_text = "Heart Prediction Dataset"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/camille-wilkens/AZ_Capstone/main/heart_failure_clinical_records_dataset.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


## Review the Dataset Result

In [8]:
dataset.take(5).to_pandas_dataframe()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [58]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 5,
     "n_cross_validations": 25,
     "primary_metric" : 'AUC_weighted'
}
 #   "primary_metric" : 'AUC_weighted'
# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="Survived",   
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [59]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)


Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
ml-experiment-1,AutoML_0b3305fc-f780-4a61-9cc7-d0140c08eef0,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


In [60]:

RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [61]:

remote_run.wait_for_completion(show_output=True)

Experiment,Id,Type,Status,Details Page,Docs Page
ml-experiment-1,AutoML_0b3305fc-f780-4a61-9cc7-d0140c08eef0,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       DONE
DESCRIPTION:  If the missing values are expected, let the run complete. Otherwise cancel the current run and use a script to customize the handling of missing feature values that may be more appropriate based on the data type and business requirement.
      

{'runId': 'AutoML_0b3305fc-f780-4a61-9cc7-d0140c08eef0',
 'target': 'automl-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-04-06T19:47:58.305273Z',
 'endTimeUtc': '2022-04-06T20:02:54.435483Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 20 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '25',
  'target': 'automl-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"ml-experiment-1","subscription_id":"6971f5ac-8af1-446e-8034-05acea24681f","resource_group":"aml-quickstarts-191197","workspace_name":"quick-starts-ws-191197","region":"southcentralus","compute_target":"aut

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [62]:
best_run,fitted_model = remote_run.get_output()
best_run_metrics = best_run.get_metrics() 

print("Best Run:",best_run.id)
print(best_run)
print("Fitted Model:", fitted_model)
print("Fitted Model Steps:")
print(fitted_model.steps)
print("Best Run Metrics")
print(best_run_metrics)

for i in best_run_metrics:
    x = best_run_metrics[i]
    print("\n", i,x )


Best Run: AutoML_0b3305fc-f780-4a61-9cc7-d0140c08eef0_38
Run(Experiment: ml-experiment-1,
Id: AutoML_0b3305fc-f780-4a61-9cc7-d0140c08eef0_38,
Type: azureml.scriptrun,
Status: Completed)
Fitted Model: Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('17', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=True)), ('extratreesclassifier', ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight='balanced', criterion='gini', max_depth=None, max_features=None, max_leaf_nodes=None, max_sa

## Test Model

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [49]:
## Register Model

from azureml.core import Model

automl_model = remote_run.register_model(model_name='Titantic_Model')
automl_model.download(exist_ok = True)

'model.pkl'

TODO: In the cell below, send a request to the web service you deployed to test it.

In [50]:
## Create Inference Config
myenv = Environment.from_conda_specification(file_path = 'titanic.yml',name='env')


TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
